In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
cols = ['SalePrice', 'Id','LotFrontage', 'Neighborhood', 'MasVnrArea', 'ExterQual',
       'BsmtQual', 'KitchenQual', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish',
       'OverallQual', 'YearBuilt', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea',
       'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea']
df = pd.read_csv("../../input/train.csv")
df = df[cols]
qualitative = list(df.columns[df.dtypes == "object"])
#qualitative += [x for x in df.select_dtypes(include=['float64',"int64"]).columns if len(df[x].unique()) <= 10]
#quantitative = [x for x in df.select_dtypes(include=['float64',"int64"]).columns if len(df[x].unique()) > 10]
quantitative = df.select_dtypes(include=['float64',"int64"]).columns
df[qualitative] = df[qualitative].fillna("None")
df[quantitative] = df[quantitative].fillna(0)

In [8]:
df = pd.read_csv("../../input/train.csv")
df.PoolQC

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
1455    NaN
1456    NaN
1457    NaN
1458    NaN
1459    NaN
Name: PoolQC, Length: 1460, dtype: object

In [7]:
#spearman correlation graphing uses the groupby of saleprice for categorical
key = pd.DataFrame(columns=["index"], index=pd.MultiIndex.from_tuples([], names=['variable','value']))
qualDf = df[qualitative]
for x in qualitative:
    kf = df.groupby(x)["SalePrice"].mean().to_frame().sort_values("SalePrice").reset_index().reset_index().drop(columns=["SalePrice"])
    kf["variable"] = x
    key = pd.concat([key, kf.rename(columns={x:"value"}).set_index(["variable","value"])])

reshape_df = pd.melt(df, id_vars="Id", value_vars=qualitative).merge(key, on=["variable","value"])
qualDf= reshape_df.pivot(index='Id', columns='variable')["index"]
df = qualDf.merge(df[quantitative], on=["Id"])




In [5]:
qualDf

variable,BsmtQual,ExterQual,FireplaceQu,GarageFinish,KitchenQual,Neighborhood
Id,,,,,,
1,3,2,1,2,2,16
2,3,1,3,2,1,20
3,3,2,3,2,2,16
4,2,1,4,1,2,17
5,3,2,3,2,2,24
...,...,...,...,...,...,...
1456,3,1,3,2,1,14
1457,3,1,3,1,1,13
1458,2,3,4,2,2,17


In [215]:
df[qualitative] = df[qualitative].astype("int64")

In [216]:
X = df.drop(columns=["SalePrice","Id"])
y = df.SalePrice

In [217]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [218]:
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import r2_score
# Instantiation 
xgb_r = xg.XGBRegressor(n_estimators = 100, seed = 123, max_depth=3, learning_rate=.08, colsample_bytree=1)
#col sample is how many they choose for tree like random forest
  
# Fitting the model 
xgb_r.fit(x_train, y_train) 
  
# Predict the model 
predictions = xgb_r.predict(x_test) 

# RMSE Computation 
rmse = np.sqrt(MSE(y_test, predictions)) 
print("RMSE : % f" %(rmse)) 

print("Test score:")
r2_score(y_test, predictions)

RMSE :  32171.573006
Test score:


0.8650630909701265

In [219]:
quantitative = quantitative[1:]
cols = cols[1:]

In [220]:
tf = pd.read_csv("../input/test.csv")
tf = tf.sort_values(by='Id')
tf = tf[cols]
tf[qualitative] = tf[qualitative].fillna("None")
tf[quantitative] = tf[quantitative].fillna(0)
reshape_tf = pd.melt(tf, id_vars="Id", value_vars=qualitative).merge(key, on=["variable","value"])
qualTf= reshape_tf.pivot(index='Id', columns='variable')["index"]
tf = qualTf.merge(tf[quantitative], on=["Id"])

tf = tf.fillna(0)

In [196]:
#key.loc[key.index.get_level_values(0)=="KitchenQual"]

In [221]:
Xtest = tf[list(X.columns)]

In [222]:
predictions = xgb_r.predict(Xtest)

In [223]:
pd.DataFrame({'SalePrice':predictions},tf.Id).to_csv("../output/XGBoost_submission.csv")